The neighbor passed the rug along to a woman he met on Tinder.

> Thankfully, this woman I met on Tinder came over at **5am** with her bike chain repair kit and some pastries from Noah’s. Apparently she liked to get up before dawn and **claim the first pastries that came out of the oven**.

So I need to find a woman who was the first to order pastries one day before 5am.
If there's multiple people who fit the criteria, I'll have to see who does it habitually/the most.

In [5]:
import polars as pl

customers = pl.read_csv("harder-data/noahs-customers.csv",try_parse_dates=True)
orders_items = pl.read_csv("harder-data/noahs-orders_items.csv",try_parse_dates=True)
orders = pl.read_csv("harder-data/noahs-orders.csv",try_parse_dates=True)
products = pl.read_csv("harder-data/noahs-products.csv",try_parse_dates=True)

It seems that the sku in the "products" table corresponds the department of the store it's from.
In this case, bakery items start with 'BKY'.

In [6]:
bakery_items = products.filter(pl.col("sku").str.starts_with("BKY"))

Now I'll find the customer who places bakery orders before 5am the most often.

In [7]:
bakery_items_orders = orders_items.join(bakery_items, on="sku", how="inner")
bakery_orders = orders.join(bakery_items_orders, on="orderid", how="inner")

early_bird_customer_id = (
    bakery_orders.filter(pl.col("ordered").dt.hour() < 5)
    .group_by("customerid")
    .agg(pl.len())
    .sort("len", descending=True)
    .limit(1)
    .select("customerid")
    .item()
)

print(
    customers.filter(pl.col("customerid") == early_bird_customer_id).select(
        "name", "phone"
    )
)


shape: (1, 2)
┌─────────────────┬──────────────┐
│ name            ┆ phone        │
│ ---             ┆ ---          │
│ str             ┆ str          │
╞═════════════════╪══════════════╡
│ Brittany Harmon ┆ 716-789-4433 │
└─────────────────┴──────────────┘


I'll admit, I pretty much was able to copy that bit straight out of the [Polars aggregation docs](https://docs.pola.rs/user-guide/expressions/aggregation/) :D